# Plymouth High School Graduation Prediction Project

## 1. Introduction

This project analyzes student data, including course history, attendance records, and state testing scores, to predict high school graduation outcomes using machine learning models.

## 2. Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

## 3. Loading, Exploring, & Cleaning Course History Data

In [ ]:
# Load Course History Excel file
df = pd.read_excel('Plymouth Data/Plymouth-Grades-Final-Excel-2.XLSX') 

In [ ]:
# Display first few rows 
df.head()

In [ ]:
# Define a mapping of letter grades to numerical values for analysis
grade_mapping = {
    'A': 4, 'B': 3, 'C': 2, 'D': 1, 'F': 0, 
    'NG': 0, 'NP': 0, 'I': 0, 'WF': 0, 'P': 0, 'M': 0
}

# Define a mapping of letter grades to numerical values for analysis
df['Mark_Numerical'] = df['Mark'].map(grade_mapping).fillna(1)  # Default to 1 for unknown marks

In [ ]:
# Specify the columns to check for duplicates
duplicate_columns = ['Student ID', 'Calendar Month', 'Calendar Year', 'Course ID']

# Find duplicate rows based on the specified columns
duplicates = df[df.duplicated(subset=duplicate_columns, keep=False)]
print("Duplicate rows based on specified columns:")
print(duplicates)

In [ ]:
# Remove duplicate rows, keeping only the first occurrence
df_cleaned = df.drop_duplicates(subset=duplicate_columns, keep='first')

# Display the cleaned DataFrame
print("\nDataFrame after removing duplicates:")
print(df_cleaned)

In [ ]:
# Create function to add student course attempted and completed
def process_student_data_with_totals(df_cleaned, middle_school_grades, high_school_grades):
    # Filter data based on grade levels
    filtered_df = df_cleaned[df_cleaned['Grade'].isin(middle_school_grades + high_school_grades)]
    
    # Aggregate to ensure uniqueness of index (Student ID and Name)
    aggregated_df = filtered_df.groupby(['Student ID', 'Student Name', 'Course Title']).agg({
        'Calendar Month': 'first',
        'Calendar Year': 'first',
        'Course ID': 'first',
        'Grade': 'first',
        'Credit Attempted': 'sum',  # Sum the credits
        'Credit Completed': 'sum',  # Sum the credits
        'School Year': 'first',
        'Mark_Numerical': 'first'
    }).reset_index()

    # Create Course Index to pivot by
    aggregated_df['Course Index'] = (
        aggregated_df.groupby(['Student ID', 'Course Title']).cumcount().astype(str) + '_' + aggregated_df['Course Title']
    )
    
    # Pivot the table so that each course detail becomes its own set of columns
    pivoted_data = aggregated_df.pivot(
        index=['Student ID', 'Student Name'],
        columns='Course Index',
        values=[
            'Calendar Month', 'Calendar Year', 'Course ID', 'Grade', 
            'Course Title', 'Credit Attempted', 'Credit Completed', 'School Year', 'Mark_Numerical'
        ]
    )
    
    # Flatten the multi-level column index
    pivoted_data.columns = [f'{col[0]}_{col[1].replace(" ", "_")}' for col in pivoted_data.columns]
    
    # Reset index to make 'Student ID' and 'Student Name' regular columns
    pivoted_data = pivoted_data.reset_index()
    
    # Identify relevant columns for middle school and high school credit attempted/completed
    credit_attempted_columns = [col for col in pivoted_data.columns if 'Credit_Attempted' in col]
    credit_completed_columns = [col for col in pivoted_data.columns if 'Credit_Completed' in col]
    
    # Ensure numeric data
    pivoted_data[credit_attempted_columns + credit_completed_columns] = pivoted_data[
        credit_attempted_columns + credit_completed_columns
    ].apply(pd.to_numeric, errors='coerce')
    
    # Recalculate totals
    pivoted_data['Total_Credit_Attempted'] = pivoted_data[credit_attempted_columns].sum(axis=1, skipna=True)
    pivoted_data['Total_Credit_Completed'] = pivoted_data[credit_completed_columns].sum(axis=1, skipna=True)
    
    # Calculate middle school and high school totals
    middle_school_df = df_cleaned[df_cleaned['Grade'].isin(middle_school_grades)]
    high_school_df = df_cleaned[df_cleaned['Grade'].isin(high_school_grades)]
    
    middle_school_credits = middle_school_df.groupby('Student ID')[['Credit Attempted', 'Credit Completed']].sum().rename(
        columns={
            'Credit Attempted': 'Middle_School_Credit_Attempted',
            'Credit Completed': 'Middle_School_Credit_Completed'
        }
    )
    high_school_credits = high_school_df.groupby('Student ID')[['Credit Attempted', 'Credit Completed']].sum().rename(
        columns={
            'Credit Attempted': 'High_School_Credit_Attempted',
            'Credit Completed': 'High_School_Credit_Completed'
        }
    )
    
    # Merge back into the pivoted data
    pivoted_data = pivoted_data.merge(middle_school_credits, on='Student ID', how='left')
    pivoted_data = pivoted_data.merge(high_school_credits, on='Student ID', how='left')
    
    # Fill NaN values for these columns with 0
    pivoted_data.fillna({
        'Middle_School_Credit_Attempted': 0,
        'Middle_School_Credit_Completed': 0,
        'High_School_Credit_Attempted': 0,
        'High_School_Credit_Completed': 0
    }, inplace=True)
    
    return pivoted_data

# Grade Levels for Middle School and High School
middle_school_grades = ['06', '07', '08', '08H']
high_school_grades = ['09', '10', '11', '12']

# Process the data
processed_data = process_student_data_with_totals(df_cleaned, middle_school_grades, high_school_grades)

# Add columns to indicate if students met credit requirements
processed_data['Middle_School_Completed'] = processed_data['Middle_School_Credit_Completed'] >= 12
processed_data['High_School_Completed'] = processed_data['High_School_Credit_Completed'] >= 24

# Determine graduation status based only on high school credits
processed_data['Graduation_Status'] = processed_data['High_School_Completed'].map({True: 'Graduated', False: 'Not Graduated'})

# Display the first few rows with the updated columns
processed_data[['Student ID', 'Middle_School_Credit_Attempted', 'Middle_School_Credit_Completed', 'High_School_Credit_Attempted', 'High_School_Credit_Completed', 'Middle_School_Completed', 'High_School_Completed', 'Graduation_Status']].head()

In [ ]:
# Specify the Student ID you're interested in
specific_student_id = 306891  # Replace with the actual Student ID you're interested in

# Filter the DataFrame to get the row corresponding to the specific Student ID
student_row = processed_data[processed_data['Student ID'] == specific_student_id]

# Print the Student ID, Credit Attempted, and Credit Completed
print(student_row[['Student ID', 'High_School_Credit_Attempted', 'High_School_Credit_Completed']])

## 4. Loading & Exploring Attendance Data

In [ ]:
# Load Attendance Excel file
df_attendance = pd.read_csv('Plymouth Data/Plymouth-Attendance-Final.CSV') 

In [ ]:
# Display first few rows
df_attendance.head()

In [ ]:
# Keep years that fall within certain range
years_to_keep = {str(year) for year in range(2008, 2022)}

# Filter rows where 'Year' is in the set of individual years
filtered_df_attendance = df_attendance[~df_attendance['School Year'].isin(years_to_keep)]

filtered_df_attendance

In [ ]:
# Pivot to make one row per student
pivoted_df_attendance = filtered_df_attendance.pivot_table(
    index=['Student ID', 'Name'],
    columns='School Year',
    values=['Grade', 'Days Absent', 'Days Membership', 'OrganizationName'],
    aggfunc='first'
).reset_index()

# Flatten multi-level column names
pivoted_df_attendance.columns = ['_'.join(filter(None, col)).strip() for col in pivoted_df_attendance.columns.to_flat_index()]

# Rename columns
pivoted_df_attendance.rename(columns={'Name': 'Student Name'}, inplace=True)
pivoted_df_attendance.head()

# Display the result
pivoted_df_attendance

In [ ]:
# Ordinals dictionary for grades, including Kindergarten
ordinals = {
    0: "Kindergarten",
    1: "1st grade", 2: "2nd grade", 3: "3rd grade",
    4: "4th grade", 5: "5th grade", 6: "6th grade",
    7: "7th grade", 8: "8th grade", 9: "9th grade",
    10: "10th grade", 11: "11th grade", 12: "12th grade"
}

# Function to map absences to grade levels based on last available year
def map_absences_by_grade(data, absent_columns):
    result = []
    for index, row in data.iterrows():
        # Extract year from column names
        year_columns = [(col, int(re.search(r'\d{4}', col).group())) for col in absent_columns]
        
        # Identify the last non-NaN year column for this student (senior year)
        last_year_col = next((col for col, year in reversed(year_columns) if not pd.isna(row[col])), None)
        senior_year = int(re.search(r'\d{4}', last_year_col).group()) if last_year_col else None
        
        grade_absences = {ordinals[grade] + ' Absences': 0 for grade in range(0, 13)}  # Include Kindergarten and "Absences"
        if senior_year:
            for col, year in year_columns:
                grade_level = 12 - (senior_year - year)  # Map years back to grade levels
                if 0 <= grade_level <= 12:
                    grade_name = ordinals[grade_level] + ' Absences'  # Add "Absences" suffix
                    grade_absences[grade_name] = row[col] if not pd.isna(row[col]) else 0

        # Include the Student ID in the result
        result.append([row['Student ID']] + list(grade_absences.values()))
    
    # Create a DataFrame from the result and set column names
    absences_by_grade_df = pd.DataFrame(result, columns=['Student ID'] + [ordinals[grade] + ' Absences' for grade in range(0, 13)])
    return absences_by_grade_df

# Extract relevant columns for absences
absent_columns = [col for col in pivoted_df_attendance.columns if "Absent" in col]

# Map the absences to grade levels
absences_by_grade_df = map_absences_by_grade(pivoted_df_attendance, absent_columns)

# Display the result
absences_by_grade_df

## 5. Loading & Exploring State Testing Data

In [ ]:
# Load State Testing Excel file
df_tests = pd.read_excel('Plymouth Data/Plymouth-Tests-Final-Excel.xlsx')

In [ ]:
# Display first few rows
df_tests.head()

In [ ]:
# Load second State Testing Excel file
df_tests2 = pd.read_excel('Plymouth Data/Plymouth-Tests-13-16-Excel.xlsx')

In [ ]:
# Display first few rows
df_tests2.head()

In [ ]:
# Merge the two data frames
merged_df = pd.concat([df_tests, df_tests2], ignore_index=True)

In [ ]:
# Pivot the original data frame to keep "Student ID" and "Scale Score"
pivoted_df_tests = pd.pivot_table(
    merged_df,
    index=['Student ID'],  # Rows grouped by unique student IDs
    columns=['Test Name', 'Part Description'],  # Pivoting by test-related columns
    values='Scale Score',  # Keeping only "Scale Score" values
    aggfunc='first'  # Assuming 'first' to handle duplicates; use 'max' or 'mean' if appropriate
)

# Flatten multi-level columns into readable column names
pivoted_df_tests.columns = ['_'.join(map(str, col)).strip() for col in pivoted_df_tests.columns.to_flat_index()]

# Reset index to return a regular DataFrame
pivoted_df_tests = pivoted_df_tests.reset_index()

# Ensure that all columns except 'Student ID' have 'Score' in their title
# Loop through the columns and add 'Score' if it's not already there
pivoted_df_tests.columns = [
    col if 'Score' in col else f"{col}_Score" if col != 'Student ID' else col
    for col in pivoted_df_tests.columns
]

# Display the pivoted DataFrame
pivoted_df_tests.head()

## 6. Merge Course History, Attendance, & State Testing Data Frames

In [ ]:
# Merge the data sets on 'Student ID'
# Start by merging grades and attendance
df_final = pd.merge(processed_data, absences_by_grade_df, on='Student ID', how='inner')

# Then merge with state_tests
final_data = pd.merge(df_final, pivoted_df_tests, on='Student ID', how='inner')

# Display the final result
final_data.head()

In [ ]:
# Check for duplicate Student IDs
duplicate_rows = final_data[final_data.duplicated(subset='Student ID', keep=False)]

# Display the duplicate rows
if not duplicate_rows.empty:
    print("Duplicate Student IDs found:")
    print(duplicate_rows)
else:
    print("No duplicate Student IDs found.")

In [ ]:
# Drop duplicate Student IDs, keeping the first occurrence
final_data_cleaned = final_data.drop_duplicates(subset='Student ID', keep='first')

# Display the cleaned DataFrame
final_data_cleaned.head()

In [ ]:
# Define the list of keywords
keywords = ["Student ID", "Numeric", "Kindergarten", "1st grade", "2nd grade", "3rd grade", "4th grade", "5th grade", "6th grade", "7th grade", "8th grade", "9th grade", "10th grade", "11th grade", "12th grade" , "Score", "Completed"]

# Filter columns that contain any of the keywords in their name
filtered_columns = [col for col in final_data_cleaned.columns if any(keyword in col for keyword in keywords)]

# Create a new DataFrame with only the filtered columns
final_data_filtered = final_data_cleaned[filtered_columns]

# Display the filtered DataFrame
final_data_filtered.head()

## 7. Data Analysis

In [ ]:
# Clean all string values in the DataFrame
final_data_filtered = final_data_filtered.applymap(lambda x: str(x).strip() if isinstance(x, str) else x)

# Replace empty strings with NaN to avoid issues
final_data_filtered.replace('', float('nan'), inplace=True)

# Define the target column (update if 'High_School_Completed' differs)
target_column = 'High_School_Completed'

# Convert target column to binary if necessary
if final_data_filtered[target_column].dtype == 'object':
    final_data_filtered[target_column] = final_data_filtered[target_column].map({'True': 1, 'False': 0})

# Define feature columns containing relevant keywords
keywords = ["Numeric", "Absences", "Score"]
features = [
    col for col in final_data_filtered.columns
    if any(keyword in col for keyword in keywords) and col != target_column
]

# Extract feature matrix and target
X = final_data_filtered[features].apply(pd.to_numeric, errors='coerce')
y = final_data_filtered[target_column]

# Drop columns with over 90% missing values
missing_percentage = X.isnull().mean()
X = X.loc[:, missing_percentage <= 0.9]

# Handle missing values
X.fillna(X.mean(), inplace=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

# Feature importance analysis
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Display top 20 features
top_20_features = feature_importances.head(20)
print("\nTop 20 Feature Importances:")

# Shorten feature names by removing everything after a closing parenthesis
top_20_features['Feature'] = top_20_features['Feature'].apply(lambda x: x.rpartition(')')[0] if ')' in x else x)

# Show the updated top 20 features
print(top_20_features)

In [ ]:
# Clean all string values in the DataFrame
final_data_filtered = final_data_filtered.applymap(lambda x: str(x).strip() if isinstance(x, str) else x)

# Replace empty strings with NaN to avoid issues
final_data_filtered.replace('', float('nan'), inplace=True)

# Define the target column (update if 'High_School_Completed' differs)
target_column = 'High_School_Completed'

# Convert target column to binary if necessary
if final_data_filtered[target_column].dtype == 'object':
    final_data_filtered[target_column] = final_data_filtered[target_column].map({'True': 1, 'False': 0})

# Define feature columns containing relevant keywords
keywords = ["Numeric", "Absences", "Score"]
features = [
    col for col in final_data_filtered.columns
    if any(keyword in col for keyword in keywords) and col != target_column
]

# Extract feature matrix and target
X = final_data_filtered[features].apply(pd.to_numeric, errors='coerce')
y = final_data_filtered[target_column]

# Drop columns with over 90% missing values
missing_percentage = X.isnull().mean()
X = X.loc[:, missing_percentage <= 0.9]

# Handle missing values
X.fillna(X.mean(), inplace=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

# Feature importance analysis
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Shorten feature names for easier visualization
def shorten_feature_name(name):
    """Shorten feature names by truncating after '(' and removing special characters."""
    name = name.split('(')[0].strip()  # Remove text after the first '('
    name = ''.join([c if c.isalnum() or c == '_' else '' for c in name])  # Remove special characters
    return name[:20]  # Limit to 20 characters for readability

# Generate feature mapping
feature_mapping = {col: shorten_feature_name(col) for col in X.columns}
shortened_feature_importances = feature_importances.copy()
shortened_feature_importances['Feature'] = shortened_feature_importances['Feature'].map(feature_mapping)

# Reverse mapping for later use
inverse_mapping = {v: k for k, v in feature_mapping.items()}

# Display top 20 features with shortened names
top_20_features = shortened_feature_importances.head(20)
print("\nTop 20 Feature Importances:")
print(top_20_features)

# Select data for the top 20 features
top_features = X[top_20_features['Feature'].map(inverse_mapping)]
correlation_matrix = top_features.corr()

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', square=True)
plt.title('Correlation Heatmap of Top 20 Features')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Define separate feature sets
numeric_features = [col for col in features if "Numeric" in col]
absent_features = [col for col in features if "Absences" in col]
score_features = [col for col in features if "Score" in col]

# Dictionary to store top features for each category
top_features_by_category = {}

# Corrected function with additional NaN handling
def analyze_features(feature_subset, category_name):
    X_subset = final_data_filtered[feature_subset].apply(pd.to_numeric, errors='coerce')
    
    # Handle missing values again to ensure no NaNs remain
    X_subset = X_subset.fillna(X_subset.mean())  # Fill remaining NaNs with column mean
    
    # Check for any remaining NaN columns and drop them if necessary
    X_subset = X_subset.dropna(axis=1, how='any')  # Drop any columns still containing NaNs
    
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3, random_state=42)
    
    # Train the Random Forest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = rf_model.predict(X_test)
    print(f"Accuracy for {category_name}: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred, zero_division=0))
    
    # Feature importance analysis
    feature_importances = pd.DataFrame({
        'Feature': X_subset.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    
    # Display top 10 features
    top_10_features = feature_importances.head(10)
    print(f"\nTop 10 {category_name} Feature Importances:")
    print(top_10_features)
    
    # Store top 10 features in dictionary
    top_features_by_category[category_name] = top_10_features
    return top_10_features

# Analyze each feature set
top_10_numeric = analyze_features(numeric_features, "Numeric")
top_10_absent = analyze_features(absent_features, "Absences")
top_10_score = analyze_features(score_features, "Score")

In [ ]:
# Extracting the top two features for absences and scores
absence_features = top_10_absent['Feature'].iloc[:2]  # Assuming 'Feature' column contains the feature names
score_features = top_10_score['Feature'].iloc[:2]     # Assuming 'Feature' column contains the feature names

# Match the pattern for OSA Reading K/S, Grade 03 in the feature names
osa_pattern = r"OSA Reading K/S, Grade 03"  # This pattern can be expanded to handle variations

# Match the pattern in the feature names (case-insensitive and allowing partial matches)
osa_features = top_10_score['Feature'].str.contains(osa_pattern, case=False, regex=True)

# Filter features based on matches
matched_osa_features = top_10_score[osa_features]

# Update your score_features to include the matched OSA features (flexible matching)
score_features = matched_osa_features['Feature'].iloc[:2]  # Adjust to ensure you have the top 2 features including OSA

# Filtering relevant data from final_data_filtered using feature names
# Remove rows with NaN in any relevant columns for absences
absence_data = final_data_filtered[list(absence_features) + ['High_School_Completed']].dropna()
X_absences = absence_data[absence_features]
y_absences = absence_data['High_School_Completed']

# Remove rows with NaN in any relevant columns for scores
score_data = final_data_filtered[list(score_features) + ['High_School_Completed']].dropna()
X_scores = score_data[score_features]
y_scores = score_data['High_School_Completed']

# Part 1: Analyzing Absences Thresholds
log_reg_absences = LogisticRegression()
log_reg_absences.fit(X_absences, y_absences)

# Define thresholds for absences (students who exceed these values are more likely to not graduate)
max_grade_12_absences = 15
max_grade_11_absences = 15

# Determine probabilities across a range of absences for Grade 12 and Grade 11
absences_range = np.linspace(0, 15, 100)  # Range of absence values
absences_probabilities = {
    'Grade 12 Absences': [],
    'Grade 11 Absences': []
}

for grade_12_abs in absences_range:
    prob_12 = log_reg_absences.predict_proba(
        pd.DataFrame([[grade_12_abs, 0]], columns=absence_features)  # Grade 11 absence is 0 for now
    )[0][1]
    absences_probabilities['Grade 12 Absences'].append((grade_12_abs, prob_12))

for grade_11_abs in absences_range:
    prob_11 = log_reg_absences.predict_proba(
        pd.DataFrame([[0, grade_11_abs]], columns=absence_features)  # Grade 12 absence is 0 for now
    )[0][1]
    absences_probabilities['Grade 11 Absences'].append((grade_11_abs, prob_11))

# Find thresholds where probability of graduation drops below 50% for both features
def find_threshold(probabilities, threshold=0.5):
    thresholds = [(value, prob) for value, prob in probabilities if prob < threshold]
    return min(thresholds, key=lambda x: x[1]) if thresholds else None

grade_12_absence_threshold = find_threshold(absences_probabilities['Grade 12 Absences'])
grade_11_absence_threshold = find_threshold(absences_probabilities['Grade 11 Absences'])

# Display the result for absences
print(f"Grade 12 Absences Threshold (Absence, Probability): {grade_12_absence_threshold}")
print(f"Grade 11 Absences Threshold (Absence, Probability): {grade_11_absence_threshold}")

In [ ]:
# Filter columns based on the specific test titles
specific_test_columns = final_data_filtered.columns[final_data_filtered.columns.str.contains('OSA Reading K/S, Grade 03 \(1B\), 2005_Demonstrate General Understanding_Score|SBA MA HS_Communicating Reasoning_Score', case=False)]

# Add the target variable 'High_School_Completed' to the list of selected columns
test_columns = list(specific_test_columns) + ['High_School_Completed']

# Filter the DataFrame to include only the relevant columns
filtered_data = final_data_filtered[test_columns]

# Check for missing values in the filtered data
print(filtered_data.isnull().sum())

# Handle missing data (e.g., fill with mean or drop rows)
filtered_data_filled = filtered_data.fillna(filtered_data.mean())  # Or drop rows with dropna() if preferred

# Verify if there are any missing values left
print(filtered_data_filled.isnull().sum())

# Define your features (X) and target variable (y)
X_testing = filtered_data_filled[specific_test_columns]
y_testing = filtered_data_filled['High_School_Completed']

# Split the data into training and testing sets
X_train_testing, X_test_testing, y_train_testing, y_test_testing = train_test_split(X_testing, y_testing, test_size=0.2, random_state=42)

# Instantiate the logistic regression model
log_reg_model = LogisticRegression(random_state=42)

# Fit the model
log_reg_model.fit(X_train_testing, y_train_testing)

# Evaluate the model
y_pred_testing = log_reg_model.predict(X_test_testing)

# Print evaluation metrics
print("Classification Report:")
print(classification_report(y_test_testing, y_pred_testing))

# Obtain the predicted probabilities for the logistic regression model
probabilities = log_reg_model.predict_proba(X_test_testing)[:, 1]  # Get the probabilities for the positive class (graduation)

# Calculate the absolute difference between predicted probabilities and 50%
probability_diff = abs(probabilities - 0.5)

# Find the indices of the closest probabilities to 50%
closest_indices = probability_diff.argsort()[:10]  # Get the top 10 closest

# Get the corresponding test scores and probabilities
closest_scores = X_test_testing.iloc[closest_indices]
closest_probs = probabilities[closest_indices]

# Combine the test scores and probabilities into a DataFrame for easy viewing
results = pd.DataFrame(closest_scores)
results['Predicted Probability'] = closest_probs

# Display the results
print("\nScores closest to a 50% probability of graduating:")
print(results)

In [ ]:
# Filter columns based on the specific test titles
specific_test_columns = final_data_filtered.columns[final_data_filtered.columns.str.contains('OSA Reading K/S, Grade 03 \(1B\), 2005_Demonstrate General Understanding_Score|SBA MA HS_Communicating Reasoning_Score', case=False)]

# Add the target variable 'High_School_Completed' to the list of selected columns
test_columns = list(specific_test_columns) + ['High_School_Completed']

# Filter the DataFrame to include only the relevant columns
filtered_data = final_data_filtered[test_columns]

# Check for missing values in the filtered data
print(filtered_data.isnull().sum())

# Handle missing data (e.g., fill with mean or drop rows)
filtered_data_filled = filtered_data.fillna(filtered_data.mean())  # Or drop rows with dropna() if preferred

# Verify if there are any missing values left
print(filtered_data_filled.isnull().sum())

# Define your features (X_scoring) and target variable (y_scoring)
X_scoring = filtered_data_filled[specific_test_columns]
y_scoring = filtered_data_filled['High_School_Completed']

# Split the data into training and testing sets
X_train_scoring, X_test_scoring, y_train_scoring, y_test_scoring = train_test_split(X_scoring, y_scoring, test_size=0.2, random_state=42)

# Instantiate the logistic regression model with class weights
log_reg_model = LogisticRegression(random_state=42, class_weight='balanced')

# Fit the model
log_reg_model.fit(X_train_scoring, y_train_scoring)

# Evaluate the model
y_pred_scoring = log_reg_model.predict(X_test_scoring)

# Print evaluation metrics
print("Classification Report:")
print(classification_report(y_test_scoring, y_pred_scoring))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test_scoring, y_pred_scoring)
print("\nConfusion Matrix:")
print(conf_matrix)

# Obtain the predicted probabilities for the logistic regression model
probabilities_scoring = log_reg_model.predict_proba(X_test_scoring)[:, 1]  # Get the probabilities for the positive class (graduation)

# Calculate the absolute difference between predicted probabilities and 50%
probability_diff_scoring = abs(probabilities_scoring - 0.5)

# Find the indices of the closest probabilities to 50%
closest_indices_scoring = probability_diff_scoring.argsort()[:10]  # Get the top 10 closest

# Get the corresponding test scores and probabilities
closest_scores_scoring = X_test_scoring.iloc[closest_indices_scoring]
closest_probs_scoring = probabilities_scoring[closest_indices_scoring]

# Combine the test scores and probabilities into a DataFrame for easy viewing
results_scoring = pd.DataFrame(closest_scores_scoring)
results_scoring['Predicted Probability'] = closest_probs_scoring

# Display the results
print("\nScores closest to a 50% probability of graduating:")
print(results_scoring)


## 8. Data Visualization

In [ ]:
# Define the three categories
numeric_features = [col for col in X.columns if "Numeric" in col]
absent_features = [col for col in X.columns if "Absences" in col]
score_features = [col for col in X.columns if "Score" in col]

# Calculate total importance for each category
total_importance_numeric = feature_importances.loc[feature_importances['Feature'].isin(numeric_features), 'Importance'].sum()
total_importance_absent = feature_importances.loc[feature_importances['Feature'].isin(absent_features), 'Importance'].sum()
total_importance_score = feature_importances.loc[feature_importances['Feature'].isin(score_features), 'Importance'].sum()

# Combine into a dictionary for easier visualization
category_importances = {
    'Numeric': total_importance_numeric,
    'Absences': total_importance_absent,
    'Score': total_importance_score
}

# Display category importance breakdown
print("\nTotal Importance by Category:")
for category, importance in category_importances.items():
    print(f"{category}: {importance:.4f}")

# Plot the importance breakdown
plt.figure(figsize=(8, 6))
plt.bar(category_importances.keys(), category_importances.values(), color=['skyblue', 'lightcoral', 'lightgreen'])
plt.title('Feature Importance Breakdown by Category')
plt.xlabel('Category')
plt.ylabel('Total Feature Importance')
plt.show()

In [ ]:
# Compute confusion matrix
cm1 = confusion_matrix(y_test, y_pred)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(cm1, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Graduated', 'Graduated'], yticklabels=['Not Graduated', 'Graduated'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Plot the top 20 feature importances
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=top_20_features, palette='viridis')
plt.title('Top 20 Feature Importances')
plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate cumulative importance
top_20_features['Cumulative Importance'] = top_20_features['Importance'].cumsum()

# Plot cumulative importance
plt.figure(figsize=(8, 6))
plt.plot(range(1, 21), top_20_features['Cumulative Importance'], marker='o', linestyle='--', color='b')
plt.xticks(range(1, 21), top_20_features['Feature'], rotation=90)
plt.title('Cumulative Feature Importance')
plt.xlabel('Top Features')
plt.ylabel('Cumulative Importance')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizing distribution of top 20 features
plt.figure(figsize=(8, 6))
sns.histplot(top_20_features['Importance'], bins=10, kde=True, color='teal')
plt.title('Distribution of Top 20 Feature Importances')
plt.xlabel('Feature Importance')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Visualizing top features for a given category and saving the chart
def plot_top_features(category, save_path=None):
    top_features = top_features_by_category[category]
    
    # Normalize feature importances for color mapping
    importance_values = top_features['Importance'].values
    norm = plt.Normalize(min(importance_values), max(importance_values))
    colors = cm.viridis(norm(importance_values))  # Use viridis colormap

    plt.figure(figsize=(10, 6))
    plt.barh(top_features['Feature'], top_features['Importance'], color=colors)
    plt.xlabel('Feature Importance')
    plt.title(f'Top 10 Features for {category}')
    plt.gca().invert_yaxis()
    
    plt.show()

# Visualize top Numeric features and save chart
plot_top_features('Numeric', save_path='top_features_numeric.png')

In [ ]:
# Visualize top Absent features
plot_top_features('Absences', save_path='top_features_absences.png')

In [ ]:
# Visualize top Score features
plot_top_features('Score', save_path='top_features_absences.png')

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_test_scoring, y_pred_scoring)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Graduated', 'Graduated'], yticklabels=['Not Graduated', 'Graduated'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Extracting the top two features for absences and scores
absence_features = top_10_absent['Feature'].iloc[:2]
score_features = top_10_score['Feature'].iloc[:2] 

# Match the pattern for OSA Reading K/S, Grade 03 in the feature names
osa_pattern = r"OSA Reading K/S, Grade 03"  # This pattern can be expanded to handle variations

# Match the pattern in the feature names (case-insensitive and allowing partial matches)
osa_features = top_10_score['Feature'].str.contains(osa_pattern, case=False, regex=True)

# Filter features based on matches
matched_osa_features = top_10_score[osa_features]

# If there are multiple matches, you can inspect or select the first one
#print("Matched OSA Features:")
#print(matched_osa_features)

# Update your score_features to include the matched OSA features (flexible matching)
score_features = matched_osa_features['Feature'].iloc[:2]

# Filtering relevant data from final_data_filtered using feature names
grade_absence_columns = ['8th grade Absences', '9th grade Absences', '10th grade Absences', '11th grade Absences', '12th grade Absences']

# Ensure relevant columns and target are present
columns_to_use = grade_absence_columns + ['High_School_Completed']
attendance_data = final_data_filtered[columns_to_use].dropna()

# Splitting data by grade and modeling each one
logistic_regressions = {}
probabilities_by_grade = {}
absences_range = np.linspace(0, attendance_data[grade_absence_columns].max().max(), 100)

for grade in grade_absence_columns:
    X = attendance_data[[grade]]  # Use attendance column for the specific grade
    y = attendance_data['High_School_Completed']
    log_reg = LogisticRegression()
    log_reg.fit(X, y)
    logistic_regressions[grade] = log_reg
    probabilities_by_grade[grade] = log_reg.predict_proba(absences_range.reshape(-1, 1))[:, 1]

# Plotting
plt.figure(figsize=(12, 8))
colors = ['blue', 'orange', 'green', 'red', 'purple']

for grade, color in zip(grade_absence_columns, colors):
    plt.plot(absences_range, probabilities_by_grade[grade], label=f"{grade.replace('_', ' ')}", color=color, linewidth=2)

plt.axhline(y=0.5, color='black', linestyle='--', label='50% Probability Threshold')
plt.title("Days Attended vs Probability of Graduation (Grades 8-12)")
plt.xlabel("Number of Days Attended")
#plt.xlim(0, 90)
plt.ylabel("Probability of Graduation")
plt.legend(loc="upper right")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Define the number of rows and columns for the grid
n_rows = 2
n_cols = 3

# Create a figure with a grid of subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 8))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop through each grade and plot on the corresponding subplot
for idx, grade in enumerate(grade_absence_columns):
    # Scatter plot
    sns.scatterplot(data=attendance_data, x=grade, y='High_School_Completed', alpha=0.6, ax=axes[idx])

    # Prepare data for logistic regression
    X = attendance_data[[grade]]  # Attendance data
    y = attendance_data['High_School_Completed']  # Graduation outcome
    
    # Fit logistic regression
    model = LogisticRegression()
    model.fit(X, y)
    
    # Create a range of values for attendance to plot the logistic curve
    x_range = np.linspace(X[grade].min(), X[grade].max(), 300).reshape(-1, 1)
    
    # Predict probabilities for the logistic curve
    y_prob = model.predict_proba(x_range)[:, 1]
    
    # Plot the logistic regression curve
    axes[idx].plot(x_range, y_prob, color='red', label='Logistic Curve')
    
    # Customize plot
    axes[idx].set_title(f"{grade.replace('_', ' ')} vs Graduation Outcome")
    axes[idx].set_xlabel("Days Attended")
    axes[idx].set_ylabel("Graduation (0 = No, 1 = Yes)")
    axes[idx].grid(True)
    axes[idx].legend()

# Hide the last subplot if there's an extra one
axes[-1].axis('off')

# Adjust layout to make it look nice
plt.tight_layout()
plt.show()

In [ ]:
# Create function for pairwise plot relationships
def plot_pairwise_relationships(top_features, category_name, target_column):
    selected_features = top_features['Feature'].tolist()
    # Add the correct target column to the subset
    subset_data = final_data_filtered[selected_features].copy()
    subset_data[target_column] = y  # Add target variable
    
    sns.pairplot(subset_data, hue=target_column, palette='Set2', diag_kind='kde')
    plt.suptitle(f"Pairwise Relationships in Top {category_name} Features", fontsize=16)
    plt.show()

# Pass the correct target column name
plot_pairwise_relationships(top_10_numeric, "Numeric", target_column="High_School_Completed")